In [24]:
import numpy as np
import pandas as pd
import feather

In [25]:
df = pd.read_feather('id_text_dialect.feather')
df.head()

,id,text,dialect
0,1175358310087892992,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .,IQ
1,1175416117793349632,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...,IQ
2,1175450108898565888,@KanaanRema مبين من كلامه خليجي,IQ
3,1175471073770573824,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐,IQ
4,1175496913145217024,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺,IQ


In [26]:
df.shape

(458197, 3)

In [27]:
df.dialect.nunique()

18

In [28]:
df.dialect.unique()

array(['IQ', 'LY', 'QA', 'PL', 'SY', 'TN', 'JO', 'MA', 'SA', 'YE', 'DZ',
       'EG', 'LB', 'KW', 'OM', 'SD', 'AE', 'BH'], dtype=object)

In [29]:
df['dialect'].value_counts()

EG    57636
PL    43742
KW    42109
LY    36499
QA    31069
JO    27921
LB    27617
SA    26832
AE    26296
BH    26292
OM    19116
SY    16242
DZ    16183
IQ    15497
SD    14434
MA    11539
YE     9927
TN     9246
Name: dialect, dtype: int64

In [30]:
df.id.duplicated().sum()

0

In [31]:
df.isnull().sum()

id         0
text       0
dialect    0
dtype: int64

In [32]:
df1 = df[['text', 'dialect']]
df1.head()

,text,dialect
0,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .,IQ
1,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...,IQ
2,@KanaanRema مبين من كلامه خليجي,IQ
3,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐,IQ
4,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺,IQ


In [33]:
import re
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def clean(line, tag = False):
    #Remove links, hashtags, at-mentions, mark-up, and "RT"
    line = re.sub(r"http\S+", "", line)
    line = re.sub(r"@\S+", "", line)
    line = re.sub(r"#\S+", "", line)
    line = re.sub("<[^>]*>", "", line)
    line = line.replace(" RT", "").replace("RT ", "")
    line = re.sub(emoji_pattern, "", line)

    return line

In [34]:
df1.iloc[3].text

'@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐'

In [35]:
clean(df1.iloc[3].text)

' يسلملي مرورك وروحك الحلوه'

In [36]:
df1['text'] = df1['text'].map(clean)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
df1.head()

,text,dialect
0,لكن بالنهاية .. ينتفض .. يغير .,IQ
1,يعني هذا محسوب على البشر .. حيونه ووحشيه .. و...,IQ
2,مبين من كلامه خليجي,IQ
3,يسلملي مرورك وروحك الحلوه,IQ
4,وين هل الغيبه اخ محمد,IQ


In [38]:
import sklearn 
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import random

In [39]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(df1.text, df1.dialect, test_size=0.2 )

In [40]:
ytest.value_counts()

EG    11473
PL     8848
KW     8350
LY     7251
QA     6197
JO     5751
LB     5554
BH     5304
AE     5303
SA     5277
OM     3871
DZ     3201
SY     3175
IQ     3121
SD     2864
MA     2250
YE     1994
TN     1856
Name: dialect, dtype: int64

In [41]:
vectorizer = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))

In [42]:
vectorizer.fit(xtrain)

TfidfVectorizer(max_features=100000, ngram_range=(1, 3))

In [ ]:
x_train = vectorizer.transform(xtrain)
x_test = vectorizer.transform(xtest).toarray()

In [ ]:
def batch_generator(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            counter=0
            




In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model_testing = Sequential()
model_testing.add(Dense(64, activation='relu', input_dim=100000))
model_testing.add(Dense(1, activation='sigmoid'))
model_testing.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_testing.fit_generator(generator=batch_generator(x_train, ytrain, 32),
                    epochs=2, validation_data=(x_test, ytest),
                    steps_per_epoch=x_train.shape[0]/32)

In [ ]:
print(classification_report(ytest, ytrain))

In [ ]:
accuracy_score(ytest, ytrain)